## Bayesian Model

In [ ]:
# importing the required packages

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import pymc3 as pm
from sklearn import preprocessing

: 

In [ ]:
# opeing the csv file from url link

df = pd.read_csv(r'https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/data/renfe_small.csv')
df= df.sample(frac=0.01, random_state=99)
df.head(5)

In [ ]:
# Open various file types; as variation

# opening a zip file
# with zipfile.ZipFile("C:\\Users\\HERLAB26\\Downloads\\Data_file.zip", "r",) as zip_ref:
#     '''extracting file to a directory set temporarily'''
#     zip_ref.extractall("temp")
# # print()

# # Reading a csvfile using pandas
# df1 = pd.read_csv("C:\\Users\\HERLAB26\\Downloads\\Data_file 2\\list.csv")

# df1.head()

# Reading a text document
# df2 = pd.read_fwf('C:\\Users\\HERLAB26\\Downloads\\Data_file 3\\words_file.txt', sep=" ", header=None, names=['col1','col2'])
# df2.head
# df2.fillna('--') # Filling the NAN values with (--) as a data cleaning step

In [ ]:
df.isnull().sum()/len(data)

In [ ]:
az.plot_kde(data['price'].values, rug=True)
plt.yticks([0], alpha=0);

In [ ]:
with pm.Model() as model_g:
    μ = pm.Uniform('μ', lower=0, upper=300)
    σ = pm.HalfNormal('σ', sd=10)
    y = pm.Normal('y', mu=μ, sd=σ, observed=data['price'].values)
    trace_g = pm.sample(1000, tune=1000)

In [ ]:
# The base model
eps = 1e-3

def dirichlet_expectation(alpha):
    if len(alpha.shape) == 1:
        return psi(alpha) - psi(np.sum(alpha))
    return psi(alpha) - psi(np.sum(alpha, 1))[:, np.newaxis]

class dmafp:
    def _init_(self, a_doc, a_voca, a_topic, alpha = 0.1, beta = 0.01, is_compute_bound=True):
        self.a_voca = a_voca
        self.a_topic = a_topic
        self.a_doc = a_doc
        self.alpha = alpha
        self.beta = beta 

        self._lambda = 1 * np.random.gamma(100., 1. / 100, (self.a_topic, self.a_voca))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self.expElogbeta = np.exp(self._Elogbeta)
        self.gamma_iter = 5
        self.gamma = 1 * np.random.gamma(100., 1. / 100, (self.a_topic, self.a_voca))
        
        self.is_compute_bound = is_compute_bound

In [ ]:
# Phrase modelling, for strings

import gensim

# Bigram and Trigram modelling
bigram = gensim.models.Phrases(df, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[df], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# checking the posterior summary of the parameters

az.summary(trace_g)

In [ ]:
#splitting the data into train and test sets

from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
y = df.iloc[:, :-1]

data_train = X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.8,
                                                   shuffle=True)

# Printing the train and test sets
print('X_train : ')
print(X_train.head())

print('X_test : ')
print(X_test.head())

print('y_train : ')
print(y_train.head())

print('y_test : ')
print(X_test.head())

In [ ]:
# plotting the gaussian model trace

az.plot(trace_g)

In [ ]:
# Bayesian inference results

az.plot_posterior(trace_g)

In [ ]:
# Plotting joint distribution of the parameters

az.plot_joint(trace_g, kind='kde', fill_last=False);

In [ ]:
# HIERARCHICAL MODEL

train_type_names = df.train_type.unique()
train_type_idx = df.train_type_encode.values

n_train_types = len(df.train_type.unique())

df[['train_type', 'price', 'fare_encode']].head()

with pm.Model() as hierarchical_model:
    '''global model parameters'''
    α_μ_tmp = pm.Normal('α_μ_tmp', mu=0., sd=100)
    α_σ_tmp = pm.HalfNormal('α_σ_tmp', 5.)
    β_μ = pm.Normal('β_μ', mu=0., sd=100)
    β_σ = pm.HalfNormal('β_σ', 5.)

    '''train type specific model parameters'''
    α_tmp = pm.Normal('α_tmp', mu=α_μ_tmp, sd=α_σ_tmp, shape=n_train_types)  
    # Intercept for each train type, distributed around train type mean 
    β = pm.Normal('β', mu=β_μ, sd=β_σ, shape=n_train_types)
    '''Model error'''
    eps = pm.HalfCauchy('eps', 5.)

    fare_est = α_tmp[train_type_idx] + β[train_type_idx]*df.fare_encode.values

    '''Data likelihood'''
    fare_like = pm.Normal('fare_like', mu=fare_est, sd=eps, observed=df.price)

In [ ]:
# The model Summary in hirarchy

with hierarchical_model:
    hierarchical_trace = pm.sample(2000, tune=2000, target_accept=0.9)

pm.traceplot(hierarchical_trace, var_names=['alpha_tmp'],coords={'alpha_tmp_dim_0': range(5)});